In [1]:
# import libraries
import pandas as pd
import numpy as np

# get the data from csv file to dataframe
raw_data = pd.read_csv('data/amazon_datafiniti.csv', low_memory=False)

# check column name
raw_data.columns

Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')

In [2]:
raw_data.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/revie...,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo,https://www.newegg.com/Product/Product.aspx%25...
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-10-09T00:00:00Z,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/...,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry,https://www.newegg.com/Product/Product.aspx%25...
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,https://www.newegg.com/Product/Product.aspx%25...


In [3]:
raw_data.shape

(5000, 24)

In [5]:
raw_data["is_duplicate"]= raw_data.duplicated()
raw_data.duplicated().sum()

95

In [6]:
#drop duplicate data
raw_data.drop_duplicates(keep="last", inplace=True)
raw_data.shape

(4905, 25)

In [7]:
# check unique username and goods name
user_count = raw_data['reviews.username'].unique().shape[0]
item_count = raw_data['name'].unique().shape[0]

print("user count: " + str(user_count) + ", item count: " + str(item_count))

user count: 3815, item count: 23


In [8]:
# indexing item
itemname = []
item_id = []
start_id = 1

for x in raw_data['name']:
    if itemname:
        for y in itemname:
            if x not in itemname :
                itemname.append(x)
                item_id.append(start_id)
                start_id = start_id + 1
            else:
                break
    else:
        itemname.append(x)
        item_id.append(start_id)
        start_id = start_id + 1

item_table = {
    'itemId':item_id,
    'itemname':itemname
}
item_df= pd.DataFrame(item_table)
# check the item_df
# item_df.head(50)
item_df.dtypes

itemId       int64
itemname    object
dtype: object

In [9]:
raw_data = pd.merge(raw_data, item_df, how='left',
                     left_on='name', right_on='itemname')

In [10]:
data_test = raw_data[['itemId', 'reviews.username','reviews.rating', 'reviews.text']]

In [11]:
data_test.head()
data_test.shape

(4905, 4)

In [12]:
# making dataset for testing 250 rows
data_test_filter = pd.DataFrame
data_test_filter = data_test.groupby('itemId').head(12)
data_test_filter.shape

(268, 4)

In [16]:
# to be 250 rows, delete (268-250=18 rows)
count = 1
drop_list = []
for x in data_test_filter.index:
    count +=1
    if count%11==0:
        drop_list.append(count)
len(drop_list)

24

In [17]:
# just need 18, delete the last 4 elements in list
del drop_list[-6:]

In [18]:
len(drop_list)

18

In [19]:
data_test_filter.drop(data_test_filter.index[drop_list], inplace=True)

/home/dea/Documents/projects/RecSys/recenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
data_test_filter.shape

(250, 4)

In [21]:
# indexing username
username = []
user_id = []
start_id = 1

for x in data_test_filter['reviews.username']:
    if username:
        for y in username:
            if x not in username :
                username.append(x)
                user_id.append(start_id)
                start_id = start_id + 1
            else:
                break
    else:
        username.append(x)
        user_id.append(start_id)
        start_id = start_id + 1

user_table = {
    'userId':user_id,
    'username':username
}
user_df = pd.DataFrame(user_table)
# check the user_df
# user_df

In [22]:
# merge user_df and data_test_filter
data_test_filter = pd.merge(data_test_filter, user_df, how='left',
                     left_on='reviews.username', right_on='username')

In [23]:
data_test_filter.head()

,itemId,reviews.username,reviews.rating,reviews.text,userId,username
0,1,llyyue,3,I thought it would be as big as small paper bu...,1,llyyue
1,1,Charmi,5,This kindle is light and easy to use especiall...,2,Charmi
2,1,johnnyjojojo,4,Didnt know how much i'd use a kindle so went f...,3,johnnyjojojo
3,1,Kdperry,5,I am 100 happy with my purchase. I caught it o...,4,Kdperry
4,1,Johnnyblack,5,Solid entry level Kindle. Great for kids. Gift...,5,Johnnyblack


In [24]:
data_test_filter.drop(["reviews.username", "username"], axis=1, inplace=True)

In [25]:
data_test_filter.head()

,itemId,reviews.rating,reviews.text,userId
0,1,3,I thought it would be as big as small paper bu...,1
1,1,5,This kindle is light and easy to use especiall...,2
2,1,4,Didnt know how much i'd use a kindle so went f...,3
3,1,5,I am 100 happy with my purchase. I caught it o...,4
4,1,5,Solid entry level Kindle. Great for kids. Gift...,5


In [26]:
data_test_filter = data_test_filter[['itemId','userId','reviews.rating', 'reviews.text']]

In [27]:
data_test_filter.head()

,itemId,userId,reviews.rating,reviews.text
0,1,1,3,I thought it would be as big as small paper bu...
1,1,2,5,This kindle is light and easy to use especiall...
2,1,3,4,Didnt know how much i'd use a kindle so went f...
3,1,4,5,I am 100 happy with my purchase. I caught it o...
4,1,5,5,Solid entry level Kindle. Great for kids. Gift...


In [28]:
data_test_filter.rename(columns={'reviews.rating': 'rating', 'reviews.text': 'comment'}, inplace=True)

In [29]:
data_test_filter.head()

,itemId,userId,rating,comment
0,1,1,3,I thought it would be as big as small paper bu...
1,1,2,5,This kindle is light and easy to use especiall...
2,1,3,4,Didnt know how much i'd use a kindle so went f...
3,1,4,5,I am 100 happy with my purchase. I caught it o...
4,1,5,5,Solid entry level Kindle. Great for kids. Gift...


In [31]:
data_test_filter.to_csv("data/data_prepared_before_sentiment.csv")

In [58]:
# Start the recommendation 

def favoriteItem(activeUser, N):
    topItem=pd.DataFrame.sort_values(
                data[data.userId==activeUser], ['rating'], ascending=[0])[:N]
    return list(topItem.itemName)

print (favoriteItem(17,3))#3 fav item of user 17


['Amazon Kindle E-Reader 6" Wifi (8th Generation, 2016)', 'Amazon Echo Show Alexa-enabled Bluetooth Speaker with 7" Screen', 'Amazon Echo Show Alexa-enabled Bluetooth Speaker with 7" Screen']


In [12]:
userItemRatingMatrix=pd.pivot_table(data, values='rating', index=['userId'], columns=['itemId'])
def matrixFactorization(R, K, steps=10, gamma=0.001, lamda=0.02):
    N=len(R.index)#number of users
    M=len(R.columns)#number of items
    P=pd.DataFrame(np.random.rand(N,K), index=R.index)
    Q=pd.DataFrame(np.random.rand(M,K), index=R.columns)
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e=e+pow(R.loc[i,j]-np.dot(P.loc[i], Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        print (step)
        if step % 100 ==0:
            print(e)
        
    return P,Q

(P,Q)=matrixFactorization(userItemRatingMatrix.iloc[:100,:23], K=2, gamma=0.001, lamda=0.02, steps=1000)


0
3297.811013044715
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
315.24822536869
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
153.1629979488978
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
26

In [13]:

activeUser=17
predictItemRating=pd.DataFrame(np.dot(P.loc[activeUser],Q.T), index=Q.index, columns=['Ratings'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Ratings'],ascending=[0])[:3]
#we found the ratings of all movie by the active user and then sorted them to top 3

topRecommendationsItems=item_df.loc[item_df.itemId.isin(topRecommendations.index)]
print (list(topRecommendationsItems.itemname))


['Kindle Oasis E-reader with Leather Charging Cover - Merlot, 6 High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers', 'Kindle Oasis E-reader with Leather Charging Cover - Black, 6" High-Resolution Display (300 ppi), Wi-Fi - Includes Special Offers', 'Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers']
